<a href="https://colab.research.google.com/github/endiesworld/2110ACDS_T7_C_Predict/blob/main/2110ACDS_T7_starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDSA Movie Recommendation 2022

© Explore Data Science Academy

---
### Honour Code

**2110ACDS_T6**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.


  

<h2><center> EDSA Movie Recommendation 2022</h2></center>
<figure>
<center><img src ="./assets/movies.png" width = "800" height = '500'/>

*Introduction*
<p align = "justify">Recommender System is a system that seeks to predict or filter preferences according to the user’s choices. Recommender systems are utilized in a variety of areas, and in this project we will use a recommender system to recommend movies for movie lovers.


*About the problem*
<p align = "justify">PUT PROBLEM STATEMENT HERE.

*Objective*
<p align = "justify"> We aim to provide an accurate and robust solution to this problem, by providing personalised recommendations to users of this product, and generating platform affinity for the streaming services which best facilitates their audience's viewing

*Process*
<p align = "justify"> In order to achieve this objective the team will follow the process below:-

1. analyse the supplied data, identify potential errors in the data and clean the existing data set;

2. determine if additional features can be added to enrich the data set;

3. build a model that is capable of predicting how a user will rate a movie;

4. evaluate the accuracy of the best machine learning model;

5. accurately predicting how a user will rate a movie they have not yet viewed, based on their historical preferences, and

6. explain the inner working of the model to a non-technical audience.

<a id="cont"></a>

## Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Data Engineering</a>

<a href=#five>5. Modeling</a>

<a href=#six>6. Model Performance</a>

<a href=#seven>7. Model Explanations</a>

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Importing Packages ⚡ |
| :--------------------------- |
| In this section you are required to import, and briefly discuss, the libraries that will be used throughout your analysis and modelling. |

---

In [49]:

# Import comet_ml at the top of your file
# from comet_ml import Experiment

# # Create an experiment with your api key
# experiment = Experiment(
#     api_key="emBEBYBp72gW5tfeZBSGftD0Y",
#     project_name="movie-recommendation",
#     workspace="emmanuelokoro",
#     log_code = True
# )

In [50]:
# Libraries for importing and loading data
import numpy as np
import pandas as pd
import scipy as sp # <-- The sister of Numpy, used in our code for numerical efficientcy. 
import matplotlib.pyplot as plt
import seaborn as sns
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Entity featurization and similarity computation
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import TfidfVectorizer

# Libraries used during sorting procedures.
import operator # <-- Convienient item retrieval during iteration 
import heapq # <-- Efficient sorting of large lists
from time import time

# Setting global constants to ensure notebook results are reproducible

RANDOM_STATE = 42


import warnings
warnings.filterwarnings('ignore')

<a id="two"></a>
## 2. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Loading the data ⚡ |
| :--------------------------- |
| In this section you are required to load the data from the `df_train` file into a DataFrame. |

---

### 2.1 Brief description of the data



In [51]:
# load the data
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
genome_scores = pd.read_csv('./data/genome_scores.csv')
genome_tags = pd.read_csv('./data/tags.csv')
imdb_data = pd.read_csv('./data/imdb_data.csv')
links = pd.read_csv('./data/links.csv')
movies = pd.read_csv('./data/movies.csv')
# tags = pd.read_csv('./data/tags.csv')

In [52]:
# Preview train dataset
print('The Shape of the data is: ', train.shape)
train.head()

The Shape of the data is:  (10000038, 4)


,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [53]:
train['userId'].nunique()

162541

In [54]:
# Preview train dataset
print('The Shape of the data is: ', test.shape)
test.head()

The Shape of the data is:  (5000019, 2)


,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [55]:
test['movieId'].nunique()

39643

In [56]:
test.tail()

,userId,movieId
5000014,162541,4079
5000015,162541,4467
5000016,162541,4980
5000017,162541,5689
5000018,162541,7153


In [57]:
# Preview genome_scores dataset
print('The Shape of the data is: ', genome_scores.shape)
genome_scores.head()

The Shape of the data is:  (15584448, 3)


,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075


In [58]:
# Preview genome_scores dataset
print('The Shape of the data is: ', genome_tags.shape)
genome_tags.head()

The Shape of the data is:  (1093360, 4)


,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [59]:
# Preview imdb_data dataset
print('The Shape of the data is: ', imdb_data.shape)
imdb_data.head()

The Shape of the data is:  (27278, 6)


,movieId,title_cast,director,runtime,budget,plot_keywords
0,1,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


In [60]:
imdb_data.isna().sum()

movieId              0
title_cast       10068
director          9874
runtime          12089
budget           19372
plot_keywords    11078
dtype: int64

In [61]:
imdb_data = imdb_data.dropna()
imdb_data.isna().sum()

movieId          0
title_cast       0
director         0
runtime          0
budget           0
plot_keywords    0
dtype: int64

In [62]:
# Preview links dataset
print('The Shape of the data is: ', links.shape)
links.head()

The Shape of the data is:  (62423, 3)


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [63]:
# Preview movies dataset
print('The Shape of the data is: ', movies.shape)
movies.head()

The Shape of the data is:  (62423, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [64]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [65]:
# Preview tags dataset
# print('The Shape of the data is: ', tags.shape)
# tags.head()

#### Dataset summary


<a id="three"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Exploratory data analysis ⚡ |
| :--------------------------- |
| In this section, you are required to perform an in-depth analysis of all the variables in the DataFrame. |

---


### 3.1 Exploratory Data Analysis
*What is Exploratory data analysis?*
    Exploratory data analysis (EDA) is the process of analysing and investigating data sets and summarizing their main characteristics, often employing both non-graphical and graphical methods. 

*Why is conducting EDA important?*
    It aids in determining how best to manipulate data to get the required answers, expose trends, patterns, and relationships that are not readily apparent i.e. get insights into the dataset.

*How is EDA conducted?*
    EDA can be conducted in the following ways:
- **Univariate**:- \
    i. **non-graphical**:- This is simplest form of data analysis, where the data being analyzed consists of just one variable. Since it’s a single variable, it doesn’t deal with causes or relationships.\
    ii. **graphical**:- Non-graphical methods don’t provide a full picture of the data. Graphical methods are therefore required. It involves visual exploratory analysis of the data.
- Multivariate:-  \
    i. **non-graphical**:- Multivariate non-graphical EDA techniques generally show the relationship between two or more variables of the data through cross-tabulation or statistics. \
    ii. **graphical**:- Multivariate data uses graphics to display relationships between two or more sets of data. The most used graphic is a grouped bar plot or bar chart with each group representing one level of one of the variables and each bar within a group representing the levels of the other variable.
    
To achieve the above, while considering the volume of dataset for this project, we make use of this python module **pandas_profiling**

#### 3.1.1 pandas_profiling
Pandas profiling is an open source Python module with which we can quickly do an exploratory data analysis with just a few lines of code. It offers report generation for the dataset with lots of features and customizations for the report generated.

In [66]:
# Generate EDA report of train dataset
# from pandas_profiling import ProfileReport
# profile = ProfileReport(train, title="Report")
# profile


In [67]:
# Generate report for genome_scores
# profile = ProfileReport(genome_scores, title="genome_scores report")
# profile


#### summarize the above.

 **Descriptive Statistics**

>Descriptive statistics summarize the data by computing mean, median, mode, standard deviation likewise.descriptive statistics describe the dataset in a way simpler manner through;

*   The measure of central tendency 
>*  Mean:- The average value 
>*  Median:- The mid point value 
>*  Mode:- The most common value

*   Measure of spread  
>* Percentiles:- Percentiles are used in statistics to give you a number that describes the value that a given percent of the values are lower than.
>* standard deviation:-a number that describes how spread out the values are.
*  Measure of symmetry 
>* Skewness:- a measure of the asymmetry of the probability distribution of a real-valued random variable about its mean.
>>* If skewness is less than -1 or greater than 1, the distribution is highly skewed.
>>* If skewness is between -1 and -0.5 or between 0.5 and 1, the distribution is moderately skewed.
>>* If skewness is between -0.5 and 0.5, the distribution is approximately symmetric. 
*  Measure of Peakedness 
>* Kurtosis:-  a measure of relative peakedness of a probability distribution, or alternatively how heavy or how light its tails are. A standard normal distribution has kurtosis of 3 and is recognized as mesokurtic. An increased kurtosis (>3) can be visualized as a thin “bell” with a high peak whereas a decreased kurtosis corresponds to a broadening of the peak and “thickening” of the tails. Kurtosis >3 is recognized as leptokurtic and <3 as platykurtic (lepto=thin; platy=broad).
>>








In [68]:
# look at data statistics


### 3.2 Key Insights from EDA 


<a id="four"></a>
## 4. Data Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Data engineering ⚡ |
| :--------------------------- |
| In this section you are required to: clean the dataset, and possibly create new features - as identified in the EDA phase. |

---

**Unique userid**
We want to evaluate the difference between the unique userId in the Train dataset and Test dataset

In [69]:
test['userId'].nunique()

162350

In [70]:
test_case = test['userId'].nunique()
train_case = train['userId'].nunique()
print('The difference in unique userID count between train and test data set is:', (train_case - test_case))

The difference in unique userID count between train and test data set is: 191


From the above, proceed to extract these 191 userIds, that are not required for prediction

In [71]:
test_userids = test['userId'].unique().tolist()
test_userids[:20]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [72]:
#Extract rows with userId present in test userId
useful_train = train[train['userId'].isin(test_userids)]
useful_train.shape

(9997845, 4)

#### Sorting of Tables

We proceed to sort both tables( train and test ) by useId

In [73]:
# Sort train dataset by userId
useful_train.sort_values(by=['userId'], inplace= True)
useful_train.head()

,userId,movieId,rating,timestamp
5122500,1,3949,5.0,1147868678
9153002,1,1175,3.5,1147868826
6923102,1,6016,5.0,1147869090
724395,1,7323,3.5,1147869119
2805472,1,4973,4.5,1147869080


In [74]:
# Sort test dataset by userId
test.sort_values(by=['userId'], inplace= True)
test.head()

,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


#### Merging of relevant tables

At this stage, we merge both tables with other tables considered to be useful for the task at hand.
The tables we merge with are listed below:
- imdb_data
- movies


In [75]:
# Merge train table with imdb_data table 
useful_train = useful_train.merge(imdb_data, on = 'movieId', how= 'left')
useful_train.head()

,userId,movieId,rating,timestamp,title_cast,director,runtime,budget,plot_keywords
0,1,3949,5.0,1147868678,Ellen Burstyn|Jared Leto|Jennifer Connelly|Mar...,Hubert Selby Jr.,102.0,"$4,500,000",drug addiction|heroin|sex show|sex scene
1,1,1175,3.5,1147868826,Pascal Benezech|Dominique Pinon|Marie-Laure Do...,Jean-Pierre Jeunet,99.0,"FRF24,000,000",black comedy|absurd comedy|surrealist|bed
2,1,6016,5.0,1147869090,Alexandre Rodrigues|Leandro Firmino|Phellipe H...,Kátia Lund,130.0,"$3,300,000",photographer|slum|gang|brazil
3,1,7323,3.5,1147869119,Daniel Brühl|Katrin Saß|Chulpan Khamatova|Mari...,Bernd Lichtenberg,121.0,"EUR4,800,000",coma|german democratic republic|capitalism|pol...
4,1,4973,4.5,1147869080,Audrey Tautou|Mathieu Kassovitz|Rufus|Lorella ...,Guillaume Laurant,122.0,"$10,000,000",female protagonist|paris france|france|montmar...


In [76]:
# Merge test table with imdb_data table 
test = test.merge(imdb_data, on = 'movieId', how= 'left')
test.head()

,userId,movieId,title_cast,director,runtime,budget,plot_keywords
0,1,2011,NaN,NaN,NaN,NaN,NaN
1,1,4144,NaN,NaN,NaN,NaN,NaN
2,1,5767,NaN,NaN,NaN,NaN,NaN
3,1,6711,Scarlett Johansson|Bill Murray|Akiko Takeshita...,Sofia Coppola,102.0,"$4,000,000",older man younger woman relationship|lonelines...
4,1,7318,Jim Caviezel|Maia Morgenstern|Christo Jivkov|F...,Benedict Fitzgerald,127.0,"$30,000,000",suffering|torture|brutality|whipping


In [77]:
# Merge train table with movies table 
useful_train = useful_train.merge(movies, on = 'movieId', how= 'left')
useful_train.head()

,userId,movieId,rating,timestamp,title_cast,director,runtime,budget,plot_keywords,title,genres
0,1,3949,5.0,1147868678,Ellen Burstyn|Jared Leto|Jennifer Connelly|Mar...,Hubert Selby Jr.,102.0,"$4,500,000",drug addiction|heroin|sex show|sex scene,Requiem for a Dream (2000),Drama
1,1,1175,3.5,1147868826,Pascal Benezech|Dominique Pinon|Marie-Laure Do...,Jean-Pierre Jeunet,99.0,"FRF24,000,000",black comedy|absurd comedy|surrealist|bed,Delicatessen (1991),Comedy|Drama|Romance
2,1,6016,5.0,1147869090,Alexandre Rodrigues|Leandro Firmino|Phellipe H...,Kátia Lund,130.0,"$3,300,000",photographer|slum|gang|brazil,City of God (Cidade de Deus) (2002),Action|Adventure|Crime|Drama|Thriller
3,1,7323,3.5,1147869119,Daniel Brühl|Katrin Saß|Chulpan Khamatova|Mari...,Bernd Lichtenberg,121.0,"EUR4,800,000",coma|german democratic republic|capitalism|pol...,"Good bye, Lenin! (2003)",Comedy|Drama
4,1,4973,4.5,1147869080,Audrey Tautou|Mathieu Kassovitz|Rufus|Lorella ...,Guillaume Laurant,122.0,"$10,000,000",female protagonist|paris france|france|montmar...,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance


In [78]:
# Merge test table with movies table 
test = test.merge(movies, on = 'movieId', how= 'left')
test.head()

,userId,movieId,title_cast,director,runtime,budget,plot_keywords,title,genres
0,1,2011,NaN,NaN,NaN,NaN,NaN,Back to the Future Part II (1989),Adventure|Comedy|Sci-Fi
1,1,4144,NaN,NaN,NaN,NaN,NaN,In the Mood For Love (Fa yeung nin wa) (2000),Drama|Romance
2,1,5767,NaN,NaN,NaN,NaN,NaN,Teddy Bear (Mis) (1981),Comedy|Crime
3,1,6711,Scarlett Johansson|Bill Murray|Akiko Takeshita...,Sofia Coppola,102.0,"$4,000,000",older man younger woman relationship|lonelines...,Lost in Translation (2003),Comedy|Drama|Romance
4,1,7318,Jim Caviezel|Maia Morgenstern|Christo Jivkov|F...,Benedict Fitzgerald,127.0,"$30,000,000",suffering|torture|brutality|whipping,"Passion of the Christ, The (2004)",Drama


#### Merging vital colunms

For this stage, we proceed to merge columns we have considered to be important in describing the content of a movie into a new column called key_words. The columns are listed below:
- title_cast
- director
- plot_keywords
- genres

In [79]:
# Merge the columns listed above into a new column named key_words fot the train data
useful_train['key_words'] = (pd.Series(useful_train[['title_cast', 'director', 'plot_keywords', 'genres']].fillna('')
                      .values.tolist()).str.join(' '))
useful_train.head()

,userId,movieId,rating,timestamp,title_cast,director,runtime,budget,plot_keywords,title,genres,key_words
0,1,3949,5.0,1147868678,Ellen Burstyn|Jared Leto|Jennifer Connelly|Mar...,Hubert Selby Jr.,102.0,"$4,500,000",drug addiction|heroin|sex show|sex scene,Requiem for a Dream (2000),Drama,Ellen Burstyn|Jared Leto|Jennifer Connelly|Mar...
1,1,1175,3.5,1147868826,Pascal Benezech|Dominique Pinon|Marie-Laure Do...,Jean-Pierre Jeunet,99.0,"FRF24,000,000",black comedy|absurd comedy|surrealist|bed,Delicatessen (1991),Comedy|Drama|Romance,Pascal Benezech|Dominique Pinon|Marie-Laure Do...
2,1,6016,5.0,1147869090,Alexandre Rodrigues|Leandro Firmino|Phellipe H...,Kátia Lund,130.0,"$3,300,000",photographer|slum|gang|brazil,City of God (Cidade de Deus) (2002),Action|Adventure|Crime|Drama|Thriller,Alexandre Rodrigues|Leandro Firmino|Phellipe H...
3,1,7323,3.5,1147869119,Daniel Brühl|Katrin Saß|Chulpan Khamatova|Mari...,Bernd Lichtenberg,121.0,"EUR4,800,000",coma|german democratic republic|capitalism|pol...,"Good bye, Lenin! (2003)",Comedy|Drama,Daniel Brühl|Katrin Saß|Chulpan Khamatova|Mari...
4,1,4973,4.5,1147869080,Audrey Tautou|Mathieu Kassovitz|Rufus|Lorella ...,Guillaume Laurant,122.0,"$10,000,000",female protagonist|paris france|france|montmar...,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance,Audrey Tautou|Mathieu Kassovitz|Rufus|Lorella ...


In [80]:
# confrim the absense of NaN value in the key_word column for the train data
nan = useful_train['key_words'].isna().sum()
print(f' There are {nan} numbers of NaN values in the train keywords column')

 There are 0 numbers of NaN values in the train keywords column


In [81]:
# Merge the columns listed above into a new column named key_words fot the test data
test['key_words'] = (pd.Series(test[['title_cast', 'director', 'plot_keywords', 'genres']].fillna('')
                      .values.tolist()).str.join(' '))
test.head()

,userId,movieId,title_cast,director,runtime,budget,plot_keywords,title,genres,key_words
0,1,2011,NaN,NaN,NaN,NaN,NaN,Back to the Future Part II (1989),Adventure|Comedy|Sci-Fi,Adventure|Comedy|Sci-Fi
1,1,4144,NaN,NaN,NaN,NaN,NaN,In the Mood For Love (Fa yeung nin wa) (2000),Drama|Romance,Drama|Romance
2,1,5767,NaN,NaN,NaN,NaN,NaN,Teddy Bear (Mis) (1981),Comedy|Crime,Comedy|Crime
3,1,6711,Scarlett Johansson|Bill Murray|Akiko Takeshita...,Sofia Coppola,102.0,"$4,000,000",older man younger woman relationship|lonelines...,Lost in Translation (2003),Comedy|Drama|Romance,Scarlett Johansson|Bill Murray|Akiko Takeshita...
4,1,7318,Jim Caviezel|Maia Morgenstern|Christo Jivkov|F...,Benedict Fitzgerald,127.0,"$30,000,000",suffering|torture|brutality|whipping,"Passion of the Christ, The (2004)",Drama,Jim Caviezel|Maia Morgenstern|Christo Jivkov|F...


In [82]:
# confrim the absense of NaN value in the key_word column for the test data
nan = test['key_words'].isna().sum()
print(f' There are {nan} numbers of NaN values in the test keywords column')

 There are 0 numbers of NaN values in the test keywords column


#### Droping of colunms not needed

Going forward, we drop colunms we have considered not realy important for the task at hand. The columns are listed below:
- runtime
- budget
- timestamp
- title_cast
- director
- plot_keywords 
- genres'

In [83]:
# Drop the above listed columns in the train data
useful_train.drop(columns=['timestamp', 'runtime', 'budget', 'title_cast', 'director', 
                           'plot_keywords','genres'], inplace= True)
useful_train.head()

,userId,movieId,rating,title,key_words
0,1,3949,5.0,Requiem for a Dream (2000),Ellen Burstyn|Jared Leto|Jennifer Connelly|Mar...
1,1,1175,3.5,Delicatessen (1991),Pascal Benezech|Dominique Pinon|Marie-Laure Do...
2,1,6016,5.0,City of God (Cidade de Deus) (2002),Alexandre Rodrigues|Leandro Firmino|Phellipe H...
3,1,7323,3.5,"Good bye, Lenin! (2003)",Daniel Brühl|Katrin Saß|Chulpan Khamatova|Mari...
4,1,4973,4.5,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Audrey Tautou|Mathieu Kassovitz|Rufus|Lorella ...


In [88]:
# Drop the above listed columns in the test data
test.drop(columns=['runtime', 'budget', 'title_cast', 'director', 
                           'plot_keywords','genres'], inplace= True)
test.head()

,userId,movieId,title,key_words
0,1,2011,Back to the Future Part II (1989),Adventure|Comedy|Sci-Fi
1,1,4144,In the Mood For Love (Fa yeung nin wa) (2000),Drama|Romance
2,1,5767,Teddy Bear (Mis) (1981),Comedy|Crime
3,1,6711,Lost in Translation (2003),Scarlett Johansson|Bill Murray|Akiko Takeshita...
4,1,7318,"Passion of the Christ, The (2004)",Jim Caviezel|Maia Morgenstern|Christo Jivkov|F...


#### Data Formating

As can be seen in the title and key_words colunm, each enity are separated by a **|**, **(** and **)**, and this character which is a separator(delimiter) if left with the data, will affect the accuracy of our model, hence needs to be removed. 

To achieve this, we write a function called splitter, to operate on both the train and test dataset. 

In [92]:
# Remove delimeters(Separators) from string data
def splitter(df, col_list, delim, separator= ' '):
    """
        This function accepts a dataframe(df) and a list of columns(col_list), which contains the delimiter
        to be removed, it also accepts the delimiter which is to be removed
    """
    new_df = df.copy()
    
    for col in col_list:
        new_df[col] = new_df[col].str.split(delim).str.join(separator)
    
    return new_df

#### Format Train Dataset

In [93]:
# Remove Opening brackets separating title of movies and year 
useful_train = splitter(useful_train, ['title'], '(', '')
useful_train.head()

,userId,movieId,rating,title,key_words
0,1,3949,5.0,Requiem for a Dream 2000),Ellen Burstyn|Jared Leto|Jennifer Connelly|Mar...
1,1,1175,3.5,Delicatessen 1991),Pascal Benezech|Dominique Pinon|Marie-Laure Do...
2,1,6016,5.0,City of God Cidade de Deus) 2002),Alexandre Rodrigues|Leandro Firmino|Phellipe H...
3,1,7323,3.5,"Good bye, Lenin! 2003)",Daniel Brühl|Katrin Saß|Chulpan Khamatova|Mari...
4,1,4973,4.5,"Amelie Fabuleux destin d'Amélie Poulain, Le) 2...",Audrey Tautou|Mathieu Kassovitz|Rufus|Lorella ...


In [95]:
# Remove close brackets separating title of movies and year 
useful_train = splitter(useful_train, ['title'], ')','')
useful_train.head()

,userId,movieId,rating,title,key_words
0,1,3949,5.0,Requiem for a Dream 2000,Ellen Burstyn|Jared Leto|Jennifer Connelly|Mar...
1,1,1175,3.5,Delicatessen 1991,Pascal Benezech|Dominique Pinon|Marie-Laure Do...
2,1,6016,5.0,City of God Cidade de Deus 2002,Alexandre Rodrigues|Leandro Firmino|Phellipe H...
3,1,7323,3.5,"Good bye, Lenin! 2003",Daniel Brühl|Katrin Saß|Chulpan Khamatova|Mari...
4,1,4973,4.5,"Amelie Fabuleux destin d'Amélie Poulain, Le 2001",Audrey Tautou|Mathieu Kassovitz|Rufus|Lorella ...


In [96]:
# Remove delimeter from key_words colunm in train data
useful_train = splitter(useful_train, ['key_words'], '|')
useful_train.head()

,userId,movieId,rating,title,key_words
0,1,3949,5.0,Requiem for a Dream 2000,Ellen Burstyn Jared Leto Jennifer Connelly Mar...
1,1,1175,3.5,Delicatessen 1991,Pascal Benezech Dominique Pinon Marie-Laure Do...
2,1,6016,5.0,City of God Cidade de Deus 2002,Alexandre Rodrigues Leandro Firmino Phellipe H...
3,1,7323,3.5,"Good bye, Lenin! 2003",Daniel Brühl Katrin Saß Chulpan Khamatova Mari...
4,1,4973,4.5,"Amelie Fabuleux destin d'Amélie Poulain, Le 2001",Audrey Tautou Mathieu Kassovitz Rufus Lorella ...


#### Format Test Dataset

In [97]:
# Remove Opening brackets separating title of movies and year 
test = splitter(test, ['title'], '(', '')
test.head()

,userId,movieId,title,key_words
0,1,2011,Back to the Future Part II 1989),Adventure|Comedy|Sci-Fi
1,1,4144,In the Mood For Love Fa yeung nin wa) 2000),Drama|Romance
2,1,5767,Teddy Bear Mis) 1981),Comedy|Crime
3,1,6711,Lost in Translation 2003),Scarlett Johansson|Bill Murray|Akiko Takeshita...
4,1,7318,"Passion of the Christ, The 2004)",Jim Caviezel|Maia Morgenstern|Christo Jivkov|F...


In [98]:
# Remove Opening brackets separating title of movies and year 
test = splitter(test, ['title'], ')', '')
test.head()

,userId,movieId,title,key_words
0,1,2011,Back to the Future Part II 1989,Adventure|Comedy|Sci-Fi
1,1,4144,In the Mood For Love Fa yeung nin wa 2000,Drama|Romance
2,1,5767,Teddy Bear Mis 1981,Comedy|Crime
3,1,6711,Lost in Translation 2003,Scarlett Johansson|Bill Murray|Akiko Takeshita...
4,1,7318,"Passion of the Christ, The 2004",Jim Caviezel|Maia Morgenstern|Christo Jivkov|F...


In [99]:
# Remove delimeter from key_words colunm in test data
test = splitter(test, ['key_words'], '|')
test.head()

,userId,movieId,title,key_words
0,1,2011,Back to the Future Part II 1989,Adventure Comedy Sci-Fi
1,1,4144,In the Mood For Love Fa yeung nin wa 2000,Drama Romance
2,1,5767,Teddy Bear Mis 1981,Comedy Crime
3,1,6711,Lost in Translation 2003,Scarlett Johansson Bill Murray Akiko Takeshita...
4,1,7318,"Passion of the Christ, The 2004",Jim Caviezel Maia Morgenstern Christo Jivkov F...


#### Merge title to key_words column

In [103]:
# Merge the columns listed above into a new column named key_words fot the test data
test['key_words'] = (pd.Series(test[['title', 'key_words']].fillna('')
                      .values.tolist()).str.join(' '))

# Drop title column
test.drop(columns=['title'], inplace=True)
test.head()

,userId,movieId,key_words
0,1,2011,Back to the Future Part II 1989 Adventure C...
1,1,4144,In the Mood For Love Fa yeung nin wa 2000 D...
2,1,5767,Teddy Bear Mis 1981 Comedy Crime
3,1,6711,Lost in Translation 2003 Scarlett Johansson Bi...
4,1,7318,"Passion of the Christ, The 2004 Jim Caviezel M..."


In [104]:
# Merge the columns listed above into a new column named key_words fot the test data
useful_train['key_words'] = (pd.Series(useful_train[['title', 'key_words']].fillna('')
                      .values.tolist()).str.join(' '))
useful_train.drop(columns=['title'], inplace=True)
useful_train.head()

,userId,movieId,rating,key_words
0,1,3949,5.0,Requiem for a Dream 2000 Ellen Burstyn Jared L...
1,1,1175,3.5,Delicatessen 1991 Pascal Benezech Dominique Pi...
2,1,6016,5.0,City of God Cidade de Deus 2002 Alexandre Rodr...
3,1,7323,3.5,"Good bye, Lenin! 2003 Daniel Brühl Katrin Saß ..."
4,1,4973,4.5,"Amelie Fabuleux destin d'Amélie Poulain, Le 20..."


#### Dividing dataset into chunks

We shall now proceed to divide both dataset into chunks. The numbers of chunk we have chosen is 162350, which is the numbers of unique userIds we have for both dataset. We do this to enables us save these chunks into local storage system of our machines, fetch these chunks back individualy and separately processing these chunks and making prediction afterwards. This act is necessary because our machines all have limited capacity, which prevents us from processing theses dataset at once.



In [105]:
# A function that generate a list of chunks
def create_chunk_list(df, col_ref, col_val):
    """
        This function accepts a dataframe, the dataframe column and the colunm value to filter by
        It returns a new dataframe, which is a datframe where the reference column matches the passed column value.
    """
    new_df = df[df[col_ref] == col_val]
        
    return new_df
            

#### Collection of unique userIds

As can be observed from our work so far, the userId are not ordered numerically, hence, there is need to collect these userIds into a list, which we use as the extensions to the chunks to be created.

In [106]:
extentions = test['userId'].unique().tolist()

print('The total numbers of extensions is: ', len(extentions))

The total numbers of extensions is:  162350


In [107]:
extentions[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

### RUN ONCE

Only run the two immediate cells bellow once, for the entire lifecycle of the project, except you delete the directories and data created after the first run.

In [108]:
# Run ONCE
# Create and store chunk for train data
t0 = time()
for index, extention in enumerate (extentions):
    
    # Create chunk name
    chunk_name = "train_chunk_{0}".format(extention)
    
    # Create and store chunk
    globals()[chunk_name] = create_chunk_list(useful_train, 'userId', extention)
    
    # Create create directory and save chunk
    directory = './data/chunked_train_data/'+chunk_name+'.csv'
    globals()[chunk_name].to_csv(directory,index=False)
    
    # Delete chunk from global memory space
    del globals()[chunk_name]
t1 = time()

print(f'I took {(t1 - t0) / 60} to create these chunks')

I took 24.567322897911072 to create these chunks


In [109]:
# Run ONCE
# Create and store chunk for test data
t0 = time()
for index, extention in enumerate (extentions):
    # Create chunk name
    chunk_name = "test_chunk_{0}".format(extention)
    
    # Create and store chunk
    globals()[chunk_name] = create_chunk_list(test, 'userId', extention)
    
    # Create create directory and save chunk
    directory = './data/chunked_test_data/'+chunk_name+'.csv'
    globals()[chunk_name].to_csv(directory,index=False)
    
    # Delete chunk from global memory space
    del globals()[chunk_name]
t1 = time()

print(f'I took {(t1 - t0) / 60} to create these chunks')

I took 13.82291675011317 to create these chunks


<a id="five"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Modelling ⚡ |
| :--------------------------- |
| In this section, you are required to create one or more regression models that are able to accurately predict the thee hour load shortfall. |

---

### 5.2 Content-based Filtering 
Making recommendations based on how similar the properties or features of an item are to other items

In [48]:
def content_generate_rating_estimate(k=4, threshold=0.0):
    frames = []
    for index, extention in enumerate (extentions):
        new_ids = []
        t0 = time()
        
        # Create chunk name
        chunk_name = "prediction_{0}".format(extention)
        directory = './data/prediction3/'+chunk_name+'.csv'
        
        
        rating_data = './data/chunked_train_data/train_chunk_{0}.csv'.format(extention)
        user = './data/chunked_test_data/test_chunk_{0}.csv'.format(extention)
        
        rating_data = pd.read_csv(rating_data)
        user = pd.read_csv(user)
        
        user_record = rating_data.drop(columns = ['rating'])
        
        
        
        combined_movies_table = pd.concat([user_record, user], ignore_index=True)
        
        titles = combined_movies_table['movieId']
        indices = pd.Series(combined_movies_table.index, combined_movies_table['movieId'])
        
        tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2),
                     min_df=0, stop_words='english')

        # Produce a feature matrix, where each row corresponds to a book,
        # with TF-IDF features as columns 
        tf_authTags_matrix = tf.fit_transform(combined_movies_table['key_words'])
        
        cosine_sim_authTags = cosine_similarity(tf_authTags_matrix, 
                                        tf_authTags_matrix)
        
        interested_movies = user['movieId'].tolist()
        
        new_ids = []
        predictedRatings = []
        for i, movie in enumerate(interested_movies):
            
            b_idx = indices[movie]
            neighbors = [] # <-- Stores our collection of similarity values 
        
            for j, row in rating_data.iterrows():
                sim = cosine_sim_authTags[b_idx-1, indices[row['movieId']]-1]
                
                neighbors.append((sim, row['rating']))
            
            # Select the top-N values from our collection
            k_neighbors = heapq.nlargest(k, neighbors, key=lambda t: t[0])
            
            # Compute the weighted average using similarity scores and 
            # user item ratings. 
            simTotal, weightedSum = 0, 0
            
            for (simScore, rating) in k_neighbors:
                # Ensure that similarity ratings are above a given threshold
                if (simScore > threshold):
                    simTotal += simScore
                    weightedSum += simScore * rating
            try:
                predictedRating = weightedSum / simTotal
            except ZeroDivisionError:
                # Cold-start problem - No ratings given by user. 
                # We use the average rating for the reference item as a proxy in this case 
                predictedRating = np.mean(rating_data[rating_data['movieId']==movie]['rating'])
            
            # Prepare Id colunm for submission 
            new_id = str(extention)+ '_' + str(movie)
            
            # Append the new id generated
            new_ids.append(new_id)
            
            # append the new prediction
            predictedRatings.append(predictedRating)
        
            
        df = pd.DataFrame({
        'Id': new_ids,
        'rating': predictedRatings
         })
        
        frames.append(df)
        df.to_csv(directory,index=False)
        
        t1 = time()
        print('Pridiction for chunk no: ', (index+1))
        print('Time elapsed: ', (t1-t0))
        
       
        
    return frames

In [1]:
# frames = content_generate_rating_estimate(k=4, threshold=0.0)

### Load Prediction Files

We load prediction files from our local machine

#### New Predicted files

In [3]:
# Check the numbers of predictions we have made so far
onlyfiles = next(os.walk('./data/predictions'))[2] #dir is your directory path as string
print (len(onlyfiles))

72216


In [4]:
# Confirm and delete '.Ds_Store'
onlyfiles.index('.DS_Store')

10709

In [5]:
# Delete the unwanted file '.Ds_Store' from the list
del onlyfiles[onlyfiles.index('.DS_Store')]

print('The exact length of predicted files is: ', len(onlyfiles))

The exact length of predicted files is:  72215


In [6]:
# Confrim the naming convention of the predicted files
print('The name of the first predicted file is: ', sorted(onlyfiles)[0])

The name of the first predicted file is:  prediction_1.csv


In [7]:
# Load all predictions 
predictions = []
for i, file_name in enumerate(onlyfiles):
    direcory = './data/predictions/'+file_name
    df = pd.read_csv(direcory)
    predictions.append(df)

# Print the length of prediction loaded
print('The length of predictions loaded is: ', len(predictions))

The length of predictions loaded is:  72215


In [8]:
# View the first prediction dataframe
predictions[0].head()

,Id,rating
0,65746_34319,2.923165
1,65746_44191,2.340489
2,65746_57640,2.889391
3,65746_33004,2.618487
4,65746_66934,2.600922


In [9]:
# Merge the prediction files
combined_predic_table = pd.concat(predictions, ignore_index=True)

print('Prediction shape is: ', combined_predic_table.shape)
combined_predic_table.head()

Prediction shape is:  (2222268, 2)


,Id,rating
0,65746_34319,2.923165
1,65746_44191,2.340489
2,65746_57640,2.889391
3,65746_33004,2.618487
4,65746_66934,2.600922


#### Old Prediction File

In [10]:
# Load old prediciton file
old_prediction = pd.read_csv('file_001.csv')
print('old_prediction shape is: ', old_prediction.shape)
old_prediction.head()

old_prediction shape is:  (5000019, 2)


,Id,rating
0,1_2011,4.0
1,1_4144,4.0
2,1_5767,4.0
3,1_6711,4.0
4,1_7318,3.0


#### Delete old prediction in new prediction file

In [11]:
# Collect new prediction Ids
new_ids = combined_predic_table['Id'].tolist()

# Drop rows with ids in new ids
old_prediction.drop(old_prediction.index[old_prediction['Id'].isin(new_ids)], inplace=True)
print('old_prediction shape is: ', old_prediction.shape)
old_prediction.head()

old_prediction shape is:  (2777751, 2)


,Id,rating
2222268,72315_6,3.0
2222269,72315_7,3.0
2222270,72315_14,3.0
2222271,72315_19,3.0
2222272,72315_29,3.0


In [45]:
# Combine old and new prediction files
complete_table = pd.concat([combined_predic_table,old_prediction], ignore_index=True)
print('The shape of the complete table is: ', complete_table.shape)
complete_table['rating'] = complete_table['rating'].round(2)
complete_table.head()

The shape of the complete table is:  (5000019, 2)


,Id,rating
0,65746_34319,2.92
1,65746_44191,2.34
2,65746_57640,2.89
3,65746_33004,2.62
4,65746_66934,2.60


In [46]:
complete_table = complete_table.fillna(3.5)

In [47]:
complete_table.isna().sum()

Id        0
rating    0
dtype: int64

In [48]:
complete_table.iloc[1262]

Id        61246_922
rating         3.99
Name: 1262, dtype: object

In [49]:
complete_table.iloc[1993]

Id        5705_175661
rating            3.5
Name: 1993, dtype: object

In [50]:

complete_table.to_csv('file_004.csv', index=False)

### 4.2 Collaborative filtering

### 4.3 Rating Prediction

As motivated previously, in some cases we may wish to directly calculate what rating a user _would_ give a book that they haven't read yet. 

We can modify our content-based filtering algorithm to do this in the following manner: 

   1. Select a reference user from the database and a reference item (movie) they have _not_ rated. 
   2. For the user, gather the similarity values between the reference item and each item the user _has_ rated. 
   3. Sort the gathered similarity values in descending order. 
   4. Select the $k$ highest similarity values which are above a given threshold value, creating a collection $K$. 
   5. Compute a weighted average rating from these values, which is the sum of the similarity values of each item multiplied by its assigned user-rating, divided by the sum of the similarity values. This can be expressed in formula as:
   
   $$ \hat{R}_{ju} = \frac{\sum_{i \in K} s_{ij} \times r_{iu}}{\sum_{i \in K} s_{ij}}   $$
   
   where $\hat{R}_{ju}$ is the weighted average computed for the reference item $j$ and reference user $u$, $K$ is the collection of items, $s_{ij}$ is the similarity computed between items $i$ and $j$, and $r_{iu}$ is the known rating user $u$ has given item $i$.
   6. We return the weighted average $\hat{R}_{ju}$ as the prediction for our reference item.
   
   
We implement this algorithmic process in the function below:

<a id="six"></a>
## 6. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model performance ⚡ |
| :--------------------------- |
| In this section you are required to compare the relative performance of the various trained ML models on a holdout dataset and comment on what model is the best and why. |

---

In [ ]:
# Compare model performance

In [ ]:
# Choose best model and motivate why it is the best choice

<a id="seven"></a>
## 7. Model Explanations
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model explanation ⚡ |
| :--------------------------- |
| In this section, you are required to discuss how the best performing model works in a simple way so that both technical and non-technical stakeholders can grasp the intuition behind the model's inner workings. |

---

In [ ]:
# discuss chosen methods logic

Run the next cell to make sure the experiment as ended. It notifies comit.

In [ ]:
experiment.end()